数据库 dase_intro_2020 有一张名为 bicycle_train 的表，该表存储了两个城市在不同因素影响下某
小时共享单车租用的数量。该表所有值均为整型数据，表结构为：
id 记录编号，无其他意义
city 城市代号，0为北京，1为上海
hour 小时，代表时间
is_workday 是否为工作日，0为否，1为是
temp_air 大气温度，单位为摄氏度
temp_body 体感温度，单位为摄氏度
weather 天气代号，1为晴天，2为多云或阴天，3为雨天或雪天
wind 风级，数值越大代表风速越大
y 该小时内共享单车被租用的数量

In [9]:
import pymysql
import numpy as np

In [10]:
db = pymysql.connect(host="cdb-r2g8flnu.bj.tencentcdb.com", port=10209,
                     user="dase2020", password="dase2020", database="dase_intro_2020")
cursor = db.cursor()  # 使用 cursor() 方法创建一个游标对象 cursor,执行SQL语句都是通过游标对象实现

1. 从第18条记录开始的5条记录；

In [11]:
sql = "SELECT * FROM bicycle_train LIMIT 17,5;"
cursor.execute(sql)
result = cursor.fetchall()
print(result)

((18, 1, 17, 1, 8, 4, 1, 2, 75), (19, 1, 1, 1, 18, 19, 1, 3, 5), (20, 1, 7, 0, 23, 25, 2, 1, 13), (21, 0, 9, 0, 25, 27, 1, 0, 67), (22, 0, 16, 1, 11, 11, 3, 2, 76))


2. 数据表中风级(wind)取值范围是多少；

In [12]:
sql = "SELECT MAX(wind),MIN(wind) FROM bicycle_train"
cursor.execute(sql)
result = cursor.fetchone()
print(f"取值范围为: {result[1]}~{result[0]}")

取值范围为: 0~7


3. 满足城市为北京市，10时，晴天，无风或1级风，租用单车数量不小于100条件下大气温度的平均
值；

In [13]:
sql = "SELECT AVG(temp_air) FROM bicycle_train WHERE city=0 AND hour=10 AND weather=1 AND (wind=0 OR wind=1) AND y>=100"
cursor.execute(sql)
result = cursor.fetchone()
print(f"大气温度为{result[0]}")

大气温度为20.6000


4. 满足城市为北京市，10时，晴天，无风或1级风，租用单车数量不小于100条件下大气温度的方
差；

In [14]:
sql = "SELECT temp_air FROM bicycle_train WHERE city=0 AND hour=10 AND weather=1 AND (wind=0 OR wind=1) AND y>=100"
cursor.execute(sql)
result = cursor.fetchall()
a = np.array(result)
lst = list(map(lambda x: int(x), a))
Var = np.var(lst)
print(f"大气温度方差为: {Var}")

大气温度方差为: 37.64


5. 分城市显示工作日雨雪天单车租用总量，并降序排序；

In [15]:
city_dic = {0: "北京", 1: "上海"}
sql = "SELECT city,SUM(y) FROM bicycle_train WHERE is_workday=1 AND weather=3 GROUP BY city ORDER BY SUM(y) DESC"
cursor.execute(sql)
result = cursor.fetchall()
print(f"{city_dic[result[0][0]]}工作日雨雪天单车租用量为{result[0][1]}, "
      f"{city_dic[result[1][0]]}工作日雨雪天单车租用量为{result[1][1]}")

上海工作日雨雪天单车租用量为9106, 北京工作日雨雪天单车租用量为8084


6. 分别查询17时至19时每小时上海市在工作日且体感温度不大于10摄氏度时租用单车的平均值（四
舍五入至整数），并且结果按升序排序。

In [16]:
sql = '''SELECT hour,AVG(y) FROM bicycle_train WHERE city=1 AND is_workday=1 AND temp_body<=10 GROUP BY hour HAVING hour
         BETWEEN 17 AND 19 ORDER BY AVG(y)'''
cursor.execute(sql)
result = cursor.fetchall()
print(f"{result[0][0]}时{round(result[0][1])}辆, {result[1][0]}时{round(result[1][1])}辆, "
      f"{result[2][0]}时{round(result[2][1])}辆")

19时42辆, 18时63辆, 17时65辆
